In [1]:
import pandas as pd
import json

In [120]:
df_train = pd.read_parquet('scotus_eyecite_train.pqt')

In [121]:
df_train

,text,label,case_id,spans,groups
0,329 U.S. 29\n67 S.Ct. 1\n91 L.Ed. 22\nCHAMPLIN...,7,329 U.S. 29,"[{""volume"": ""329"", ""reporter"": ""U.S."", ""page"":...","[[{""volume"": ""329"", ""reporter"": ""U.S."", ""page""..."
1,329 U.S. 1\n67 S.Ct. 6\n91 L.Ed. 3\nHALLIBURTO...,7,329 U.S. 1,"[{""volume"": ""329"", ""reporter"": ""U.S."", ""page"":...","[[{""volume"": ""329"", ""reporter"": ""U.S."", ""page""..."
2,329 U.S. 14\n67 S.Ct. 13\n91 L.Ed. 12\nCLEVELA...,0,329 U.S. 14,"[{""volume"": ""329"", ""reporter"": ""U.S."", ""page"":...","[[{""volume"": ""329"", ""reporter"": ""U.S."", ""page""..."
3,329 U.S. 40\n67 S.Ct. 167\n91 L.Ed. 29\nUNITED...,1,329 U.S. 40,"[{""volume"": ""329"", ""reporter"": ""U.S."", ""page"":...","[[{""volume"": ""329"", ""reporter"": ""U.S."", ""page""..."
4,329 U.S. 90\n67 S.Ct. 133\n91 L.Ed. 103\nAMERI...,7,329 U.S. 90,"[{""volume"": ""329"", ""reporter"": ""U.S."", ""page"":...","[[{""volume"": ""329"", ""reporter"": ""U.S."", ""page""..."
...,...,...,...,...,...
4995,459 U.S. 176\n103 S.Ct. 539\n74 L.Ed.2d 348\nS...,10,459 U.S. 176,"[{""volume"": ""459"", ""reporter"": ""U.S."", ""page"":...","[[{""volume"": ""459"", ""reporter"": ""U.S."", ""page""..."
4996,459 U.S. 212\n103 S.Ct. 588\n74 L.Ed.2d 402\nC...,6,459 U.S. 212,"[{""volume"": ""459"", ""reporter"": ""U.S."", ""page"":...","[[{""volume"": ""459"", ""reporter"": ""U.S."", ""page""..."
4997,459 U.S. 297\n103 S.Ct. 634\n74 L.Ed.2d 465\nD...,7,459 U.S. 297,"[{""volume"": ""459"", ""reporter"": ""U.S."", ""page"":...","[[{""volume"": ""459"", ""reporter"": ""U.S."", ""page""..."
4998,459 U.S. 248\n103 S.Ct. 608\n74 L.Ed.2d 430\nT...,0,459 U.S. 248,"[{""volume"": ""459"", ""reporter"": ""U.S."", ""page"":...","[[{""volume"": ""459"", ""reporter"": ""U.S."", ""page""..."


In [36]:

def check_groups(row):
    groups = json.loads(row['groups'])
    spans = json.loads(row['spans'])
    def check_span(start,end,groups):
        for group in groups:
            for item in group:
                if item['start'] == start and item['end'] == end:
                    return True
        return False
    for span in spans:
        start = span['start']
        end = span['end']
        assert check_span(start, end,groups),span


def check_groups_inside_allowed(row):
    groups = json.loads(row['groups'])
    spans = json.loads(row['spans'])
    def check_span(start,end,groups):
        for group in groups:
            for item in group:
                if item['start'] <= start and item['end'] >= end:
                    return True
        return False
    for span in spans:
        start = span['start']
        end = span['end']
        assert check_span(start, end,groups),span



In [37]:



spans = json.loads(df_train.iloc[0]['spans'])
groups = json.loads(df_train.iloc[0]['groups'])
text = df_train.iloc[0]['text']


In [128]:
import eyecite
import eyecite.models

def check_row(text):
    citations = eyecite.get_citations(text)
    resolved = eyecite.resolve_citations(citations)
    s1 = sorted([c.span() for c in citations if not isinstance(c,eyecite.models.UnknownCitation)])
    s2 = sorted([c.span() for group in resolved.values() for c in group if not isinstance(c,eyecite.models.UnknownCitation)])
    return len(s1)==len(s2)


def check_row2(text):
    citations = eyecite.get_citations(text)
    resolved = eyecite.resolve_citations(citations)
    s1 = {c.span() for c in citations}
    s2 = {c.span() for group in resolved.values() for c in group}
    s1only = s1.difference(s2)
    s2only = s2.difference(s1)
    r1 = [citation for citation in citations if citation.span() in s1only]
    r2 =  [citation for group in resolved.values()  for citation in group if citation.span() in s2only]
    
    return dict(s1only=r1,s2only=r2)



In [132]:
items = df_train.text.head().map(check_row2)

,s1only,s2only
0,"[UnknownCitation('§', metadata=CitationBase.Me...",[]
1,"[UnknownCitation('§', metadata=CitationBase.Me...",[]
2,"[UnknownCitation('§', metadata=CitationBase.Me...",[]
3,"[SupraCitation('supra,', metadata=SupraCitatio...",[]
4,"[UnknownCitation('§', metadata=CitationBase.Me...",[]


In [90]:
inconsistent = df_train[~ df_train.consisten]

In [91]:
inconsistent.head()





,text,label,case_id,spans,groups,consisten
1,329 U.S. 1\n67 S.Ct. 6\n91 L.Ed. 3\nHALLIBURTO...,7,329 U.S. 1,"[{""volume"": ""329"", ""reporter"": ""U.S."", ""page"":...","[[{""volume"": ""329"", ""reporter"": ""U.S."", ""page""...",False
2,329 U.S. 14\n67 S.Ct. 13\n91 L.Ed. 12\nCLEVELA...,0,329 U.S. 14,"[{""volume"": ""329"", ""reporter"": ""U.S."", ""page"":...","[[{""volume"": ""329"", ""reporter"": ""U.S."", ""page""...",False
3,329 U.S. 40\n67 S.Ct. 167\n91 L.Ed. 29\nUNITED...,1,329 U.S. 40,"[{""volume"": ""329"", ""reporter"": ""U.S."", ""page"":...","[[{""volume"": ""329"", ""reporter"": ""U.S."", ""page""...",False
4,329 U.S. 90\n67 S.Ct. 133\n91 L.Ed. 103\nAMERI...,7,329 U.S. 90,"[{""volume"": ""329"", ""reporter"": ""U.S."", ""page"":...","[[{""volume"": ""329"", ""reporter"": ""U.S."", ""page""...",False
5,329 U.S. 64\n67 S.Ct. 154\n91 L.Ed. 44\nUNITED...,7,329 U.S. 64,"[{""volume"": ""329"", ""reporter"": ""U.S."", ""page"":...","[[{""volume"": ""329"", ""reporter"": ""U.S."", ""page""...",False


In [92]:
spans = json.loads(inconsistent.iloc[0].spans)
groups = json.loads(inconsistent.iloc[0].groups)

In [93]:
len(groups)

59

In [94]:
len(spans)

83

In [118]:
check_row2(inconsistent.iloc[0].text)

[SupraCitation('supra,', metadata=SupraCitation.Metadata(parenthetical=None, antecedent_guess='Corporation', pin_cite=None, volume=None)),
 SupraCitation('supra.', metadata=SupraCitation.Metadata(parenthetical=None, antecedent_guess='Corporation', pin_cite=None, volume=None)),
 SupraCitation('supra,', metadata=SupraCitation.Metadata(parenthetical=None, antecedent_guess='Wells', pin_cite=None, volume=None)),
 SupraCitation('supra,', metadata=SupraCitation.Metadata(parenthetical=None, antecedent_guess='Wells', pin_cite=None, volume=None)),
 SupraCitation('supra,', metadata=SupraCitation.Metadata(parenthetical=None, antecedent_guess='Corporation', pin_cite=None, volume=None)),
 SupraCitation('supra,', metadata=SupraCitation.Metadata(parenthetical=None, antecedent_guess='Yentzer', pin_cite=None, volume=None)),
 SupraCitation('supra,', metadata=SupraCitation.Metadata(parenthetical=None, antecedent_guess='Wells', pin_cite=None, volume=None))]

In [113]:
s1


{(9610, 9616),
 (11356, 11362),
 (13264, 13270),
 (14497, 14503),
 (14994, 15000),
 (18879, 18885),
 (18943, 18949)}

In [114]:
s2

set()

In [98]:
len(resolved),len(citations)

(21, 65)

In [89]:
resolved

defaultdict(list,
            {Resource(citation=FullCaseCitation('329 U.S. 29', groups={'volume': '329', 'reporter': 'U.S.', 'page': '29'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant=None, extra=None))): [FullCaseCitation('329 U.S. 29', groups={'volume': '329', 'reporter': 'U.S.', 'page': '29'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant=None, extra=None))],
             Resource(citation=FullCaseCitation('67 S.Ct. 1', groups={'volume': '67', 'reporter': 'S.Ct.', 'page': '1'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant=None, extra=None))): [FullCaseCitation('67 S.Ct. 1', groups={'volume': '67', 'reporter': 'S.Ct.', 'page': '1'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant

In [70]:
list(resolved.items())[-1]

(4,
 defaultdict(list,
             {Resource(citation=FullCaseCitation('329 U.S. 90', groups={'volume': '329', 'reporter': 'U.S.', 'page': '90'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant=None, extra=None))): [FullCaseCitation('329 U.S. 90', groups={'volume': '329', 'reporter': 'U.S.', 'page': '90'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant=None, extra=None))],
              Resource(citation=FullCaseCitation('67 S.Ct. 133', groups={'volume': '67', 'reporter': 'S.Ct.', 'page': '133'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant=None, extra=None))): [FullCaseCitation('67 S.Ct. 133', groups={'volume': '67', 'reporter': 'S.Ct.', 'page': '133'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=

defaultdict(list,
            {Resource(citation=FullCaseCitation('329 U.S. 29', groups={'volume': '329', 'reporter': 'U.S.', 'page': '29'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant=None, extra=None))): [FullCaseCitation('329 U.S. 29', groups={'volume': '329', 'reporter': 'U.S.', 'page': '29'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant=None, extra=None))],
             Resource(citation=FullCaseCitation('67 S.Ct. 1', groups={'volume': '67', 'reporter': 'S.Ct.', 'page': '1'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant=None, extra=None))): [FullCaseCitation('67 S.Ct. 1', groups={'volume': '67', 'reporter': 'S.Ct.', 'page': '1'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant

In [76]:
def span_pairs(item):
    match item:
        case dict():
            for k,v in item.items():
                source = span(k)
                for item in v:
                    yield source,span(item)
                    
        case _:
            yield item
        
        
def span(item):
    match item:
        case eyecite.models.Resource():
            return span(item.citation)
        case eyecite.models.CitationBase():
            return item.span()

In [88]:
def get_links(text):
    citations = eyecite.get_citations(text)
    resolved = eyecite.resolve_citations(citations)
    links = [dict(target_text=text[target[0]:target[1]],
              target_span=target,
              source_span=text[source[0]:source[1]],
              source=source) for target,source in span_pairs(resolved) if target != source]
    return links


In [93]:
print(json.dumps(get_links(df_train['text'][2]),indent=4))

[
    {
        "target_text": "322 U.S. 369",
        "target_span": [
            1377,
            1389
        ],
        "source_span": "322 U.S. 369",
        "source": [
            25595,
            25607
        ]
    },
    {
        "target_text": "64 S.Ct. 1037",
        "target_span": [
            1391,
            1404
        ],
        "source_span": "64 S.Ct. 1037",
        "source": [
            25609,
            25622
        ]
    },
    {
        "target_text": "88 L.Ed. 1331",
        "target_span": [
            1406,
            1419
        ],
        "source_span": "88 L.Ed. 1331",
        "source": [
            3328,
            3341
        ]
    },
    {
        "target_text": "88 L.Ed. 1331",
        "target_span": [
            1406,
            1419
        ],
        "source_span": "88 L.Ed. 1331",
        "source": [
            7523,
            7536
        ]
    },
    {
        "target_text": "88 L.Ed. 1331",
        "target_span": [
         